Author: Chris J. Dallmann \
Affiliation: University of Wuerzburg \
Last revision: 12-March-2025

In [1]:
from caveclient import CAVEclient
import numpy as np 
import pandas as pd

In [2]:
# Access FlyWire public dataset 
client = CAVEclient()
datastack_name = 'flywire_fafb_public'
client = CAVEclient(datastack_name)

In [3]:
# Set ID of web neuron 
source_ids = [720575940636656632] # Left brain

In [4]:
# Get neuron information
df_neuron = client.materialize.query_table('proofread_neurons')

In [5]:
# Get neuron classifications 
df_class = client.materialize.query_table('hierarchical_neuron_annotations', filter_in_dict={'classification_system': ['super_class']})

In [6]:
# Get presynaptic partners 
df_connectivity = client.materialize.query_view('valid_synapses_nt_np', filter_in_dict={'post_pt_root_id': source_ids})

In [7]:
# Remove connections less than syn_thresh  
syn_thresh = 5;
df_connectivity['syn_count'] = 1
df_connectivity = df_connectivity.groupby(['pre_pt_root_id','post_pt_root_id'])['syn_count'].sum().reset_index().sort_values('syn_count', ascending=False)
df_connectivity = df_connectivity[df_connectivity.syn_count >= syn_thresh]

In [8]:
# Remove IDs belonging to fragments (non-proofread neurons) 
proofread_neuron_ids = df_neuron.pt_root_id.unique().tolist()
df_connectivity = df_connectivity[df_connectivity.pre_pt_root_id.isin(proofread_neuron_ids)]

In [9]:
# Compute connectivity matrix 
pre_ids = df_connectivity.pre_pt_root_id.unique().tolist()
post_ids = df_connectivity.post_pt_root_id.unique().tolist()

connectivity_matrix = np.zeros((len(pre_ids),len(post_ids)))
for pre_index, pre_id in enumerate(pre_ids):
    df = df_connectivity[df_connectivity.pre_pt_root_id == pre_id]
    for index, row in df.iterrows():
        post_index = post_ids.index(row.post_pt_root_id)
        syn_count = row.syn_count
        connectivity_matrix[pre_index, post_index] += syn_count

In [10]:
# Get IDs of partner neurons 
partner_ids = pre_ids 

## Synapse counts per cell type

In [11]:
# Get cell types of partner neurons
partner_cell_types = []
for partner_id in partner_ids:
    partner_cell_types.append(df_class.cell_type[df_class.pt_root_id == partner_id].item())

In [12]:
# Initialize dict with synapse and neuron counts per cell type
dict_n_synapses = {}
dict_n_neurons = {}
for cell_type in list(set(partner_cell_types)):
    dict_n_synapses[cell_type] = 0
    dict_n_neurons[cell_type] = 0
    
# Populate dict based on connectivity matrix
for index, partner_id in enumerate(partner_ids):
    dict_n_synapses[partner_cell_types[index]]  += int(np.sum(connectivity_matrix[:][index])) / int(np.sum(connectivity_matrix))
    dict_n_neurons[partner_cell_types[index]] += 1

In [13]:
dict_n_synapses

{'descending': 0.2531558014871175,
 'central': 0.48037350855957145,
 'ascending': 0.26647068995331163}

In [14]:
dict_n_neurons

{'descending': 37, 'central': 59, 'ascending': 74}

## Synapse counts per neuropil

In [15]:
# Get presynaptic partners of partner neurons
df_connectivity_partner = client.materialize.query_view('valid_synapses_nt_np', filter_in_dict={'post_pt_root_id': partner_ids})

In [16]:
# Remove connections less than syn_thresh  
syn_thresh = 5;
df_connectivity_partner_summed = df_connectivity_partner
df_connectivity_partner_summed['syn_count'] = 1
df_connectivity_partner_summed = df_connectivity_partner_summed.groupby(['pre_pt_root_id','post_pt_root_id'])['syn_count'].sum().reset_index().sort_values('syn_count', ascending=False)
df_connectivity_partner_summed = df_connectivity_partner_summed[df_connectivity_partner_summed.syn_count >= syn_thresh]

keys = ['pre_pt_root_id','post_pt_root_id']
i1 = df_connectivity_partner.set_index(keys).index
i2 = df_connectivity_partner_summed.set_index(keys).index
df_connectivity_partner = df_connectivity_partner[i1.isin(i2)]

In [17]:
df_connectivity_partner

,id,pre_pt_root_id,post_pt_root_id,connection_score,cleft_score,gaba,ach,glut,oct,ser,da,valid_nt,pre_pt_supervoxel_id,post_pt_supervoxel_id,neuropil,pre_pt_position,post_pt_position,syn_count
0,501056,720575940628115561,720575940627490459,1434.721680,156,0.008052,0.653634,3.020457e-01,0.000049,3.579354e-02,0.000424,t,77691216942537904,77691216942534471,SAD,"[410568, 269088, 74560]","[410392, 269064, 74560]",1
2,226825707,720575940632308644,720575940631483695,1035.334595,148,0.005627,0.007205,9.866252e-01,0.000471,5.237106e-06,0.000067,t,79448168005590033,79448168005587296,SMP_R,"[513368, 135672, 136280]","[513248, 135640, 136320]",1
6,132442881,720575940640180797,720575940622669971,1222.673828,150,0.000070,0.997009,5.328720e-10,0.000079,2.113336e-07,0.002841,t,78042442322008872,78042442322007560,LAL_R,"[429852, 233116, 104920]","[429920, 233012, 104920]",1
7,153913557,720575940628321352,720575940622669971,257.227478,132,0.002822,0.821526,1.344984e-03,0.110296,7.462041e-05,0.063937,t,78042098657842611,77971729913667337,LAL_R,"[429792, 214992, 94840]","[429696, 214924, 94840]",1
9,99461147,720575940624481674,720575940617809181,202.215820,134,0.025286,0.442753,1.138557e-01,0.000737,1.339232e-01,0.283445,t,81278992170282912,81278992170282913,PVLP_L,"[621920, 209356, 86640]","[621996, 209424, 86640]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235047,210623148,720575940620005542,720575940620556966,24.132742,72,0.036050,0.836519,8.062545e-03,0.000161,7.008483e-02,0.049122,t,80998685491882819,80998685491894893,AMMC_L,"[602312, 277704, 109920]","[602364, 277680, 110000]",1
235048,135125693,720575940624471373,720575940610505006,90.301414,143,0.002891,0.990962,6.155567e-04,0.004696,1.493182e-05,0.000820,t,81913066715014277,81913066715004691,AVLP_L,"[656240, 254660, 66200]","[656388, 254712, 66200]",1
235049,79294325,720575940622452715,720575940622493260,225.578033,151,0.068231,0.442167,4.558906e-01,0.000756,1.006635e-02,0.022889,t,80718309892877997,80718309892873124,GNG,"[589448, 344984, 78360]","[589272, 345024, 78240]",1
235050,211522240,720575940620005542,720575940620556966,286.635864,76,0.009076,0.855628,7.732723e-03,0.001137,1.066777e-02,0.115759,t,80998616772401250,80998616772406312,SAD,"[603036, 275768, 109760]","[603076, 275800, 109800]",1


In [21]:
# Generate list of neuropil names with _L and _R
neuropil_names_merged = df_connectivity_partner.neuropil.to_list()
for index, name in enumerate(neuropil_names_merged):
    if '_' in name:
        neuropil_names_merged[index] = name[:-(len(name)-name.find('_'))] 

# Replace neuropil names 
df_connectivity_partner['neuropil'] = pd.Series(neuropil_names_merged).values
df_connectivity_partner.reset_index(drop=True, inplace=True)

# Sum entries where neuropil is the same. Keep pre_root_id and post_root_id.
df_connectivity_partner = df_connectivity_partner.groupby(['neuropil','pre_pt_root_id','post_pt_root_id'])['syn_count'].sum().reset_index()

In [22]:
# Generate dataframe with weighted synapse counts from partner neuropils
connectivity_matrix_neuropil = []

for source_index, source_id in enumerate(source_ids):
    for partner_index, partner_id in enumerate(partner_ids):
        n_syn_source_partner = int(connectivity_matrix[:][partner_index][0])
        
        if n_syn_source_partner>0:
            # post_root_ids are partner neurons, pre_root_ids are neurons presynaptic to partner neurons
            df = df_connectivity_partner[df_connectivity_partner.post_pt_root_id == partner_id]
                
            # Sum syn_count for each neuropil
            df = df.groupby(['post_pt_root_id','neuropil'])['syn_count'].sum().reset_index()
            
            # Add to data structure
            for index, row in df.iterrows():
                connectivity_matrix_neuropil.append([source_id,  
                                                     partner_id, 
                                                     df.neuropil[index], # Neuropil of partner 
                                                     int(df.syn_count[index]) / int(np.sum(df.syn_count)), # Relative syn_count of partner in neuropil
                                                     n_syn_source_partner, # syn_count between source and partner
                                                     int(df.syn_count[index]) / int(np.sum(df.syn_count)) * n_syn_source_partner]) # Weighted syn_count of partner            

# Generate dataframe 
df_connectivity_neuropil = pd.DataFrame(connectivity_matrix_neuropil)
df_connectivity_neuropil.columns = ["source_id", "partner_id", "neuropil", "rel_syn_count_partner", "syn_count_source_partner", "weighted_syn_count_partner"]      
df_connectivity_neuropil

,source_id,partner_id,neuropil,rel_syn_count_partner,syn_count_source_partner,weighted_syn_count_partner
0,720575940636656632,720575940634528864,CAN,0.000377,316,0.119155
1,720575940636656632,720575940634528864,FLA,0.036199,316,11.438914
2,720575940636656632,720575940634528864,GNG,0.880845,316,278.346908
3,720575940636656632,720575940634528864,IPS,0.003394,316,1.072398
4,720575940636656632,720575940634528864,SAD,0.069759,316,22.043741
...,...,...,...,...,...,...
814,720575940636656632,720575940613761858,SAD,0.003165,5,0.015823
815,720575940636656632,720575940613761858,VES,0.006329,5,0.031646
816,720575940636656632,720575940613761858,WED,0.015823,5,0.079114
817,720575940636656632,720575940620473547,GNG,1.000000,5,5.000000


In [23]:
# Sum weights for each neuopil
df_connectivity_neuropil_summed = df_connectivity_neuropil.groupby(['neuropil'])['weighted_syn_count_partner'].sum().reset_index().sort_values('weighted_syn_count_partner', ascending=False)
df_connectivity_neuropil_summed['rel_weighted_syn_count_partner'] = df_connectivity_neuropil_summed.weighted_syn_count_partner / df_connectivity_neuropil_summed.weighted_syn_count_partner.sum()
df_connectivity_neuropil_summed

,neuropil,weighted_syn_count_partner,rel_weighted_syn_count_partner
12,GNG,3267.473748,0.607224
4,AVLP,467.330974,0.086848
24,SAD,375.474211,0.069778
15,ICL,359.633095,0.066834
13,GOR,162.846169,0.030263
23,PVLP,151.993239,0.028246
26,SIP,73.861376,0.013726
30,VES,68.926243,0.012809
10,FLA,58.830290,0.010933
25,SCL,54.805156,0.010185
